#### Connecting Google Drive

In [ ]:
from google.colab import drive
drive.mount("gdrive/")

#### Avoids scroll-in-the-scroll in the entire Notebook

In [ ]:
from IPython.display import Javascript
def resize_colab_cell():
  display(Javascript("google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})"))
get_ipython().events.register("pre_run_cell", resize_colab_cell)

#### Preprocessing Dataset with `GloVe` encoding

In [ ]:
import numpy as np
from gensim.models import KeyedVectors

glove_model = {}
with open("/content/gdrive/MyDrive/Colab Notebooks/additional_files/glove.6B.100d.txt", "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype="float32")
        glove_model[word] = vector

import pandas as pd
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/datasets/file1.csv")

# print(df.shape)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df["Text"], df["Language"], test_size=0.2, random_state=42)

def encode_text(text):
    word_embeddings = []
    for word in text.split():
        if word in glove_model:
            word_embeddings.append(glove_model[word])
    if word_embeddings:
        return np.mean(word_embeddings, axis=0)
    else:
        return np.zeros(100)

x_train_encoded = np.vstack(x_train.apply(lambda x: encode_text(x)))
x_test_encoded = np.vstack(x_test.apply(lambda x: encode_text(x)))

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
x_train_vectorized = vectorizer.fit_transform(x_train)
x_test_vectorized = vectorizer.transform(x_test)

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

x_train_combined = np.hstack((x_train_encoded, x_train_vectorized.toarray()))
x_test_combined = np.hstack((x_test_encoded, x_test_vectorized.toarray()))

#### 1. Using `Multinomial Naive Bayes`

In [ ]:
from sklearn.naive_bayes import MultinomialNB 
MNB = MultinomialNB()
MNB.fit(x_train_combined.clip(min=0), y_train_encoded)

y_pred_MNB = MNB.predict(x_test_combined)

from sklearn.metrics import accuracy_score 
print(f"MNB & GloVe: {accuracy_score(y_test_encoded, y_pred_MNB) * 100 :.2f}%")

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_MNB = encoder.inverse_transform(y_pred_MNB)

accuracy = accuracy_score(y_test, y_pred_MNB)
precision = precision_score(y_test, y_pred_MNB, average="macro")
recall = recall_score(y_test, y_pred_MNB, average="macro")
f1 = f1_score(y_test, y_pred_MNB, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_MNB)
confusion_mat = confusion_matrix(y_test, y_pred_MNB)
class_report = classification_report(y_test, y_pred_MNB)
mcc = matthews_corrcoef(y_test, y_pred_MNB)
balanced_acc = balanced_accuracy_score(y_test, y_pred_MNB)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_MNB)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%") 

#### 2. Using `Support Vector Machine`

In [ ]:
from sklearn.svm import SVC
SVM = SVC(kernel="linear")
SVM.fit(x_train_combined, y_train_encoded)

y_pred_SVM = SVM.predict(x_test_combined)

from sklearn.metrics import accuracy_score
print(f"SVM & GloVe: {accuracy_score(y_test_encoded, y_pred_SVM) * 100 :.2f}%")

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_SVM = encoder.inverse_transform(y_pred_SVM)

accuracy = accuracy_score(y_test, y_pred_SVM)
precision = precision_score(y_test, y_pred_SVM, average="macro")
recall = recall_score(y_test, y_pred_SVM, average="macro")
f1 = f1_score(y_test, y_pred_SVM, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_SVM)
confusion_mat = confusion_matrix(y_test, y_pred_SVM)
class_report = classification_report(y_test, y_pred_SVM)
mcc = matthews_corrcoef(y_test, y_pred_SVM)
balanced_acc = balanced_accuracy_score(y_test, y_pred_SVM)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_SVM)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%")  

#### 3. Using `Random Forest`

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier()
RF.fit(x_train_combined, y_train_encoded)

y_pred_RF = RF.predict(x_test_combined)

from sklearn.metrics import accuracy_score
print(f"Random Forest & GloVe: {accuracy_score(y_test_encoded, y_pred_RF) * 100 :.2f}%")

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_RF = encoder.inverse_transform(y_pred_RF)

accuracy = accuracy_score(y_test, y_pred_RF)
precision = precision_score(y_test, y_pred_RF, average="macro")
recall = recall_score(y_test, y_pred_RF, average="macro")
f1 = f1_score(y_test, y_pred_RF, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_RF)
confusion_mat = confusion_matrix(y_test, y_pred_RF)
class_report = classification_report(y_test, y_pred_RF)
mcc = matthews_corrcoef(y_test, y_pred_RF)
balanced_acc = balanced_accuracy_score(y_test, y_pred_RF)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_MNB)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%")  

#### 4. Using `Gradient Boosting Classifier`

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier()
GB.fit(x_train_combined.clip(min=0), y_train_encoded)

y_pred_GB = GB.predict(x_test_combined)

from sklearn.metrics import accuracy_score
print(f"Gradient Boosting Classsifier & GloVe: {accuracy_score(y_test_encoded, y_pred_GB) * 100 :.2f}%")

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_GB = encoder.inverse_transform(y_pred_GB)

accuracy = accuracy_score(y_test, y_pred_GB)
precision = precision_score(y_test, y_pred_GB, average="macro")
recall = recall_score(y_test, y_pred_GB, average="macro")
f1 = f1_score(y_test, y_pred_GB, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_GB)
confusion_mat = confusion_matrix(y_test, y_pred_GB)
class_report = classification_report(y_test, y_pred_GB)
mcc = matthews_corrcoef(y_test, y_pred_GB)
balanced_acc = balanced_accuracy_score(y_test, y_pred_GB)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_GB)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%") 

#### 5. Using `Decision Tree Classifier`

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(x_train_combined, y_train_encoded)

y_pred_DT = DT.predict(x_test_combined)

from sklearn.metrics import accuracy_score
print(f"Decision Tree Classsifier & GloVe: {accuracy_score(y_test_encoded, y_pred_DT) * 100 :.2f}%")

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_DT = encoder.inverse_transform(y_pred_DT)

accuracy = accuracy_score(y_test, y_pred_DT)
precision = precision_score(y_test, y_pred_DT, average="macro")
recall = recall_score(y_test, y_pred_DT, average="macro")
f1 = f1_score(y_test, y_pred_DT, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_DT)
confusion_mat = confusion_matrix(y_test, y_pred_DT)
class_report = classification_report(y_test, y_pred_DT)
mcc = matthews_corrcoef(y_test, y_pred_DT)
balanced_acc = balanced_accuracy_score(y_test, y_pred_DT)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_DT)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%") 

#### 6. Using `K-Nearest Neighbors Classifier`

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier()
KNN.fit(x_train_combined, y_train_encoded)

y_pred_KNN = KNN.predict(x_test_combined)

from sklearn.metrics import accuracy_score
print(f"K-Nearest Neighbors Classifier & GloVe: {accuracy_score(y_test_encoded, y_pred_KNN) * 100 :.2f}%") 

In [ ]:

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_KNN = encoder.inverse_transform(y_pred_KNN)

accuracy = accuracy_score(y_test, y_pred_KNN)
precision = precision_score(y_test, y_pred_KNN, average="macro")
recall = recall_score(y_test, y_pred_KNN, average="macro")
f1 = f1_score(y_test, y_pred_KNN, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_KNN)
confusion_mat = confusion_matrix(y_test, y_pred_KNN)
class_report = classification_report(y_test, y_pred_KNN)
mcc = matthews_corrcoef(y_test, y_pred_KNN)
balanced_acc = balanced_accuracy_score(y_test, y_pred_KNN)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_KNN)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%") 

#### 7. Using `AdaBoost Classifier`

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ADB = AdaBoostClassifier()
ADB.fit(x_train_combined, y_train_encoded)

y_pred_ADB = ADB.predict(x_test_combined)

from sklearn.metrics import accuracy_score
print(f"AdaBoost Classifier & GloVe: {accuracy_score(y_test_encoded, y_pred_ADB) * 100 :.2f}%") 

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_ADB = encoder.inverse_transform(y_pred_ADB)

accuracy = accuracy_score(y_test, y_pred_ADB)
precision = precision_score(y_test, y_pred_ADB, average="macro")
recall = recall_score(y_test, y_pred_ADB, average="macro")
f1 = f1_score(y_test, y_pred_ADB, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_ADB)
confusion_mat = confusion_matrix(y_test, y_pred_ADB)
class_report = classification_report(y_test, y_pred_ADB)
mcc = matthews_corrcoef(y_test, y_pred_ADB)
balanced_acc = balanced_accuracy_score(y_test, y_pred_ADB)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_ADB)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%") 

#### 8. Using `Logistic Regression`

In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(x_train_combined, y_train_encoded)

y_pred_LR = LR.predict(x_test_combined)

from sklearn.metrics import accuracy_score
print(f"Logistic Regression & GloVe: {accuracy_score(y_test_encoded, y_pred_LR) * 100 :.2f}%") 

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_LR = encoder.inverse_transform(y_pred_LR)

accuracy = accuracy_score(y_test, y_pred_LR)
precision = precision_score(y_test, y_pred_LR, average="macro")
recall = recall_score(y_test, y_pred_LR, average="macro")
f1 = f1_score(y_test, y_pred_LR, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_LR)
confusion_mat = confusion_matrix(y_test, y_pred_LR)
class_report = classification_report(y_test, y_pred_LR)
mcc = matthews_corrcoef(y_test, y_pred_LR)
balanced_acc = balanced_accuracy_score(y_test, y_pred_LR)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_LR)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%")

#### 9. Using `Extra Trees Classifier`

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
ET = ExtraTreesClassifier()
ET.fit(x_train_combined, y_train_encoded)

y_pred_ET = ET.predict(x_test_combined)

from sklearn.metrics import accuracy_score
print(f"Extra Trees Classifier & GloVe: {accuracy_score(y_test_encoded, y_pred_ET) * 100 :.2f}%")

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_ET = encoder.inverse_transform(y_pred_ET)

accuracy = accuracy_score(y_test, y_pred_ET)
precision = precision_score(y_test, y_pred_ET, average="macro")
recall = recall_score(y_test, y_pred_ET, average="macro")
f1 = f1_score(y_test, y_pred_ET, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_ET)
confusion_mat = confusion_matrix(y_test, y_pred_ET)
class_report = classification_report(y_test, y_pred_ET)
mcc = matthews_corrcoef(y_test, y_pred_ET)
balanced_acc = balanced_accuracy_score(y_test, y_pred_ET)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_ET)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%")

#### 10. Using `Gaussian Naive Bayes`

In [ ]:
# from sklearn.gaussian_process import GaussianProcessRegressor
# GPR = GaussianProcessRegressor()

from sklearn.naive_bayes import GaussianNB
GPR = GaussianNB()
GPR.fit(x_train_combined.clip(min=0), y_train_encoded)

y_pred_GPR = GPR.predict(x_test_combined)

from sklearn.metrics import accuracy_score
print(f"Gaussian Naive Bayes & GloVe: {accuracy_score(y_test_encoded, y_pred_GPR.round()) * 100 :.2f}%")

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_GPR = encoder.inverse_transform(y_pred_GPR)

accuracy = accuracy_score(y_test, y_pred_GPR)
precision = precision_score(y_test, y_pred_GPR, average="macro")
recall = recall_score(y_test, y_pred_GPR, average="macro")
f1 = f1_score(y_test, y_pred_GPR, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_GPR)
confusion_mat = confusion_matrix(y_test, y_pred_GPR)
class_report = classification_report(y_test, y_pred_GPR)
mcc = matthews_corrcoef(y_test, y_pred_GPR)
balanced_acc = balanced_accuracy_score(y_test, y_pred_GPR)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_GPR)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%")  

#### 11. Using `Ridge Classifier`

In [ ]:
from sklearn.linear_model import RidgeClassifier
RR = RidgeClassifier()
RR.fit(x_train_combined, y_train_encoded)

y_pred_RR = RR.predict(x_test_combined)

from sklearn.metrics import accuracy_score
print(f"Ridge Classifier & GloVe: {accuracy_score(y_test_encoded, y_pred_RR) * 100:.2f}%")

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_RR = encoder.inverse_transform(y_pred_RR)

accuracy = accuracy_score(y_test, y_pred_RR)
precision = precision_score(y_test, y_pred_RR, average="macro")
recall = recall_score(y_test, y_pred_RR, average="macro")
f1 = f1_score(y_test, y_pred_RR, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_RR)
confusion_mat = confusion_matrix(y_test, y_pred_RR)
class_report = classification_report(y_test, y_pred_RR)
mcc = matthews_corrcoef(y_test, y_pred_RR)
balanced_acc = balanced_accuracy_score(y_test, y_pred_RR)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_RR)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%")

#### 12. Using `Elastic Net Regression`

In [ ]:
from sklearn.linear_model import ElasticNet
EN = ElasticNet()
EN.fit(x_train_combined, y_train_encoded)

y_pred_EN = EN.predict(x_test_combined).round()

from sklearn.metrics import accuracy_score
print(f"Elastic Net Regression & GloVe: {accuracy_score(y_test_encoded, y_pred_EN) * 100 :.2f}%")

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_EN = encoder.inverse_transform(y_pred_EN.astype(int))

accuracy = accuracy_score(y_test, y_pred_EN)
precision = precision_score(y_test, y_pred_EN, average="macro")
recall = recall_score(y_test, y_pred_EN, average="macro")
f1 = f1_score(y_test, y_pred_EN, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_EN)
confusion_mat = confusion_matrix(y_test, y_pred_EN)
class_report = classification_report(y_test, y_pred_EN)
mcc = matthews_corrcoef(y_test, y_pred_EN)
balanced_acc = balanced_accuracy_score(y_test, y_pred_EN)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_EN)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%") 

#### 13. Using `Multilayer Perceptron`

In [ ]:
from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier(max_iter=10000)
MLP.fit(x_train_combined, y_train_encoded)

y_pred_MLP = MLP.predict(x_test_combined)

from sklearn.metrics import accuracy_score
print(f"Multilayer Perceptron & GloVe: {accuracy_score(y_test_encoded, y_pred_MLP) * 100 :.2f}%")

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_MLP = encoder.inverse_transform(y_pred_MLP)

accuracy = accuracy_score(y_test, y_pred_MLP)
precision = precision_score(y_test, y_pred_MLP, average="macro")
recall = recall_score(y_test, y_pred_MLP, average="macro")
f1 = f1_score(y_test, y_pred_MLP, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_MLP)
confusion_mat = confusion_matrix(y_test, y_pred_MLP)
class_report = classification_report(y_test, y_pred_MLP)
mcc = matthews_corrcoef(y_test, y_pred_MLP)
balanced_acc = balanced_accuracy_score(y_test, y_pred_MLP)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_MLP)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%") 

#### 14. Using `Lasso Regressor`

In [ ]:
from sklearn.linear_model import Lasso
LASSO = Lasso()
LASSO.fit(x_train_combined, y_train_encoded)

y_pred_LASSO = LASSO.predict(x_test_combined)
y_pred_LASSO = [round(i) for i in y_pred_LASSO]

from sklearn.metrics import accuracy_score
print(f"Lasso Regression & GloVe: {accuracy_score(y_test_encoded, y_pred_LASSO) * 100 :.2f}%")

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_LASSO = encoder.inverse_transform(y_pred_LASSO)

accuracy = accuracy_score(y_test, y_pred_LASSO)
precision = precision_score(y_test, y_pred_LASSO, average="macro")
recall = recall_score(y_test, y_pred_LASSO, average="macro")
f1 = f1_score(y_test, y_pred_LASSO, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_LASSO)
confusion_mat = confusion_matrix(y_test, y_pred_LASSO)
class_report = classification_report(y_test, y_pred_LASSO)
mcc = matthews_corrcoef(y_test, y_pred_LASSO)
balanced_acc = balanced_accuracy_score(y_test, y_pred_LASSO)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_LASSO)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%") 

#### 15. Using `XG Boost`

In [ ]:
import xgboost as xgb
XGB = xgb.XGBClassifier()
XGB.fit(x_train_combined, y_train_encoded)

y_pred_XGB = XGB.predict(x_test_combined)

from sklearn.metrics import accuracy_score
print(f"XG Boost & GloVe: {accuracy_score(y_test_encoded, y_pred_XGB) * 100 :.2f}%") 

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_XGB = encoder.inverse_transform(y_pred_XGB)

accuracy = accuracy_score(y_test, y_pred_XGB)
precision = precision_score(y_test, y_pred_XGB, average="macro")
recall = recall_score(y_test, y_pred_XGB, average="macro")
f1 = f1_score(y_test, y_pred_XGB, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_XGB)
confusion_mat = confusion_matrix(y_test, y_pred_XGB)
class_report = classification_report(y_test, y_pred_XGB)
mcc = matthews_corrcoef(y_test, y_pred_XGB)
balanced_acc = balanced_accuracy_score(y_test, y_pred_XGB)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_XGB)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%") 

#### 16. Using `CNN`

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

x_train_reshaped_CNN = x_train_combined.reshape((x_train_combined.shape[0], x_train_combined.shape[1], 1))
x_test_reshaped_CNN = x_test_combined.reshape((x_test_combined.shape[0], x_test_combined.shape[1], 1))

CNN = Sequential()
CNN.add(Conv1D(filters=64, kernel_size=3, activation="relu", input_shape=(x_train_combined.shape[1], 1)))
CNN.add(GlobalMaxPooling1D())
CNN.add(Dense(64, activation="relu"))
CNN.add(Dropout(0.2))
CNN.add(Dense(len(encoder.classes_), activation="softmax"))

CNN.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
CNN.fit(x_train_reshaped_CNN, y_train_encoded, epochs=100, batch_size=32, validation_split=0.1)

accuracy = CNN.evaluate(x_test_reshaped_CNN, y_test_encoded)[1]
print(f"CNN Accuracy: {accuracy * 100:.2f}%")

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

y_pred_CNN = encoder.inverse_transform(np.argmax(CNN.predict(x_test_reshaped_CNN), axis=1))

accuracy = accuracy_score(y_test, y_pred_CNN)
precision = precision_score(y_test, y_pred_CNN, average="macro")
recall = recall_score(y_test, y_pred_CNN, average="macro")
f1 = f1_score(y_test, y_pred_CNN, average="macro")
kappa = cohen_kappa_score(y_test, y_pred_CNN)
confusion_mat = confusion_matrix(y_test, y_pred_CNN)
class_report = classification_report(y_test, y_pred_CNN)
mcc = matthews_corrcoef(y_test, y_pred_CNN)
balanced_acc = balanced_accuracy_score(y_test, y_pred_CNN)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)
y_pred_binary = lb.transform(y_pred_CNN)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%")

#### 17. Using `RNN (LSTM)`

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequence_length = max(len(seq.split()) for seq in x_train)  # Maximum sequence length
x_train_padded = pad_sequences(x_train_encoded, maxlen=sequence_length, padding="post")
x_test_padded = pad_sequences(x_test_encoded, maxlen=sequence_length, padding="post")

embedding_size = 100 
RNN = Sequential()
RNN.add(Embedding(len(glove_model), embedding_size, input_length=sequence_length, trainable=False))
RNN.add(LSTM(units=64))
RNN.add(Dropout(rate=0.2))
RNN.add(Dense(units=len(encoder.classes_), activation="softmax"))

RNN.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
RNN.fit(x_train_padded, y_train_encoded, epochs=100, batch_size=32, validation_split=0.1)

accuracy = RNN.evaluate(x_test_padded, y_test_encoded)[1]
print(f"RNN (LSTM) Accuracy: {accuracy * 100:.2f}%")

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score,
    roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score, matthews_corrcoef
)

x_test_reshaped_RNN = x_test_padded.reshape((x_test_padded.shape[0], sequence_length))
y_pred_RNN = np.argmax(RNN.predict(x_test_reshaped_RNN), axis=1)

accuracy = accuracy_score(y_test_encoded, y_pred_RNN)
precision = precision_score(y_test_encoded, y_pred_RNN, average="macro")
recall = recall_score(y_test_encoded, y_pred_RNN, average="macro")
f1 = f1_score(y_test_encoded, y_pred_RNN, average="macro")
kappa = cohen_kappa_score(y_test_encoded, y_pred_RNN)
confusion_mat = confusion_matrix(y_test_encoded, y_pred_RNN)
class_report = classification_report(y_test_encoded, y_pred_RNN)
mcc = matthews_corrcoef(y_test_encoded, y_pred_RNN)
balanced_acc = balanced_accuracy_score(y_test_encoded, y_pred_RNN)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test_encoded)
y_pred_binary = lb.transform(y_pred_RNN)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, average="macro")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
print(f"Cohen's Kappa: {kappa * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(class_report)
print(f"Matthews Correlation Coefficient: {mcc * 100:.2f}%")
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%")